# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [32]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json
import numpy as np




# Import API key
from api_keys import geoapify_key

In [33]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hawaiian paradise park,19.5933,-154.9731,27.80,78,0,4.12,US,1697336535
1,1,tobelo,1.7284,128.0095,29.53,72,84,2.43,ID,1697336535
2,2,brody,50.0879,25.1503,14.51,85,100,5.05,UA,1697336535
3,3,gadzhiyevo,69.2551,33.3362,2.18,97,100,15.47,RU,1697336536
4,4,adamstown,-25.0660,-130.1015,21.66,78,99,2.33,PN,1697336536


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [34]:
%%capture --no-display

# Configure the map plot
city_map_plot = city_data_df.hvplot.points(
    'Lng', 'Lat', geo=True, color='City', size = "Humidity", alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500
    )

# Display the map
city_map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [35]:
# Narrow down cities that fit criteria and drop any results with null values

# Drop any rows with null value
city_data_fc = city_data_df.dropna()
city_data_fc = city_data_fc[(city_data_fc['Max Temp'] > 21) 
             & (city_data_fc['Max Temp'] < 27)
             & (city_data_fc['Cloudiness'] == 0)
             & (city_data_fc['Wind Speed'] < 4.5)]


# Display sample data
city_data_fc.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
92,92,tura,25.5198,90.2201,24.02,72,0,0.36,IN,1697336556
96,96,butte,39.6499,-121.5677,21.01,46,0,0.00,US,1697336557
118,118,kaliganj,23.4095,89.1402,25.87,69,0,0.96,BD,1697336563
129,129,palasa,18.7667,84.4167,26.25,73,0,1.24,IN,1697336566
137,137,margaret river,-33.9500,115.0667,21.73,45,0,2.00,AU,1697336567


### Step 3: Create a new DataFrame called `hotel_df`.

In [36]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_fc[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
92,tura,IN,25.5198,90.2201,72,
96,butte,US,39.6499,-121.5677,46,
118,kaliganj,BD,23.4095,89.1402,69,
129,palasa,IN,18.7667,84.4167,73,
137,margaret river,AU,-33.9500,115.0667,45,
163,sur,OM,22.5667,59.5289,77,
192,birzebbuga,MT,35.8258,14.5269,82,
193,san antonio,US,29.4241,-98.4936,36,
271,narasannapeta,IN,18.4167,84.0500,73,
333,saint augustine beach,US,29.8505,-81.2654,83,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [38]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories":"accommodation.hotel",
          "apiKey": geoapify_key,
          "limit": 20 
         }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    
    # get latitude, longitude from the DataFrame
    
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address =  requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    #print(json.dumps(name_address, indent=4, sort_keys=True))
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
tura - nearest hotel: No hotel found
butte - nearest hotel: No hotel found
kaliganj - nearest hotel: No hotel found
palasa - nearest hotel: No hotel found
margaret river - nearest hotel: Margaret River Hotel
sur - nearest hotel: Sur Hotel
birzebbuga - nearest hotel: Seabreeze
san antonio - nearest hotel: Riverwalk Plaza Hotel and Suites
narasannapeta - nearest hotel: No hotel found
saint augustine beach - nearest hotel: Castillo Real Resort Hotel
bahawalnagar - nearest hotel: No hotel found
wenchang - nearest hotel: No hotel found
pachino - nearest hotel: Hotel Celeste
teknaf - nearest hotel: No hotel found
port isabel - nearest hotel: Queen Isabel Inn
afif - nearest hotel: Samaya Royal Afif
vieste - nearest hotel: Palace Hotel Vieste


,City,Country,Lat,Lng,Humidity,Hotel Name
92,tura,IN,25.5198,90.2201,72,No hotel found
96,butte,US,39.6499,-121.5677,46,No hotel found
118,kaliganj,BD,23.4095,89.1402,69,No hotel found
129,palasa,IN,18.7667,84.4167,73,No hotel found
137,margaret river,AU,-33.9500,115.0667,45,Margaret River Hotel
163,sur,OM,22.5667,59.5289,77,Sur Hotel
192,birzebbuga,MT,35.8258,14.5269,82,Seabreeze
193,san antonio,US,29.4241,-98.4936,36,Riverwalk Plaza Hotel and Suites
271,narasannapeta,IN,18.4167,84.0500,73,No hotel found
333,saint augustine beach,US,29.8505,-81.2654,83,Castillo Real Resort Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [39]:
%%capture --no-display

# Configure the map plot
hotel_map_plot = hotel_df.hvplot.points(
    'Lng', 'Lat', geo=True, color='City', size = "Humidity", alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500
    )


# Display the map
hotel_map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)